<a href="https://colab.research.google.com/github/gryffin5/AppRepo/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
# go to this site: https://www.tensorflow.org/tutorials/images/classification
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
tfds.disable_progress_bar()

## Access Google Drive

In [2]:
import pathlib
from google.colab import drive
drive.mount("/content/gdrive")
data_dir_training = "gdrive/My Drive/training"
data_dir_training = pathlib.Path(data_dir_training)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Prepare Data

In [5]:
batch_size = 32
img_height = 3000
img_width = 5000

In [6]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_training,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 12226 files belonging to 5 classes.
Using 9781 files for training.


In [7]:
ds_val = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_training,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 12226 files belonging to 5 classes.
Using 2445 files for validation.


In [8]:
data_dir_testing = "gdrive/My Drive/testing"
data_dir_testing = pathlib.Path(data_dir_testing)

In [10]:
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_testing,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3073 files belonging to 5 classes.


## Finish Preparation

In [ ]:
train_images = np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example['label'].numpy() for example in ds_train])

test_images = np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example['label'].numpy() for example in ds_test])

In [ ]:
train_images = train_images.reshape(2520, 5000, 3000, 1)
test_images = test_images.reshape(372, 5000, 3000, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

## Train a Convolution Network

In [ ]:
model = keras.Sequential([
   keras.layers.Conv2D(64, 3, activation='relu', input_shape=(5000,3000,1)),
   keras.layers.Conv2D(32, 3, activation='relu'),
   keras.layers.Flatten(),
   keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images,test_labels)

## Hyperparameter Tuning

In [ ]:
pip install -U keras-tuner

In [ ]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6,3,input_shape=(5000,3000,1)))

  for i in range(hp.Int("Conv Layers", min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16,32,64]), 3, activation='relu'))
  
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(hp.Choice("Dense layer", [64, 128, 256, 512, 1024]), activation='relu'))

  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  
  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=32,
)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(test_images, test_labels)

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

## Save and Load Models

In [ ]:
best_model.save('./eye_classification_model')

In [ ]:
loaded_model = keras.models.load_model('./eye_classification_model')

In [ ]:
loaded_model.evaluate(test_images, test_labels)

In [ ]:
rgb_images = np.array([example['image'].numpy() for example in ds_train.take(1)])
rgb_image = rgb_images[0]
plt.imshow(rgb_image)
rgb_image.shape

## Test Model with Single Example

In [ ]:
result = best_model.predict(np.array([train_images[0]]))
print(result)

predicted_value = np.argmax(result)
print(predicted_value)